In [1]:
import os
import cv2
import numpy as np

In [2]:
#Label untuk kelas image
subjects = ["","Female","Male"]

In [3]:
#fungsi untuk mendeteksi wajah menggunakan openCV
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)#konversi gambar dari RGB to Greyscale
    #mengakses file .xml untuk metode LBP pada oprenCV
    face_cascade = cv2.CascadeClassifier('/Users/Alwi/Desktop/DIP/Fisher face/lbpcascade_frontalface.xml')
    #mendeteksi multiscale image hasil keluaran berupa wajah yang terdeteksi
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    #if no faces are detected then return original img
    if (len(faces) == 0): #jika pada gambar tidak terdeteksi bentuk wajah maka return "none" 
        return None, None
    x, y, w, h = faces[0] #mengekstrak area dari wajah
    return gray[y:y+h, x:x+w] , faces[0] #yang diambil dari gambar hanyalah wajah 

In [4]:
def prepare_training_data(data_folder_path): #fungsi untuk mempersiapkan training data
    folder1 = os.listdir(data_folder_path) #mengakses folder yg berisi dataset training
    faces = [] #list face
    labels = [] #list label
     #mengakses gambar-gambar yg ada pada folder
    for dirs in folder1: 
        if dirs.startswith("f") or dirs.startswith("m"):
            if dirs == "female" :
                label = 1
            else:
                label = 2
            dir_path = data_folder_path + "/" + dirs
            folder2 = os.listdir (dir_path)
            for dir_name in folder2: 
                subject_dir_path = data_folder_path + "/" + dirs + "/" + dir_name
                subject_images_names = os.listdir(subject_dir_path)
                for image_name in subject_images_names:
                    if image_name.startswith("."):
                        continue;
        #image path = training-data/s1/jokowi1.jpg
                    image_path = subject_dir_path + "/" + image_name

        #membuka gambar sesuai image_path
                    image = cv2.imread(image_path)

        #menampilkan gambar
                    cv2.imshow("Training on image...", image)
                    cv2.waitKey(100)

        #memanggil fungsi deteksi wajah
                    face, rect = detect_face(image)
        
                    if face is not None: 
                        faceresize = cv2.resize(face, (100, 100)) #resize gambar training
            #membuat array face dan label
                        faces.append(faceresize)
                        labels.append(label)
        else:
            continue;
    #menutup jendela yg kebuka pada saat fungsi persiapan training data berjalan
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
 
    return faces, labels

In [5]:
def draw_rectangle(img, rect): #membuat bound box bentuk segiempat pada muka yg terdeteksi
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

def draw_text(img, text, x, y): #menampilkan text label pada muka yg terdeteksi sesuai posisi (x,y)
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [6]:
#Proses Training Data. list labels dan faces punya ukuran yg sama, list face berisi wajah, list labels berupa nama kelas
print("Preparing data...")
faces, labels = prepare_training_data("Data-Training") #memanggil fungsi untuk mempersiapkan data
print("Data prepared")
 
#melihat jumlah face dan label pada training data
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))
face_recognizer =  cv2.face.EigenFaceRecognizer_create() #melakukan face recognition menggunakan metode fisherface
face_recognizer.train(faces, np.array(labels))

Preparing data...
Data prepared
Total faces:  709
Total labels:  709


In [7]:
def testing(image):
    img = image.copy() #copy gambar agar gambar asli tidak berubah
    #mendeteksi wajah
    face, rect = detect_face(img)
    face = cv2.resize(face, (100, 100)) #resize gambar agar sesuai dengan size gambar pada training data 
    label= face_recognizer.predict(face) #prediksi wajah yang terdeteksi masuk kedalam subject jokowi/prabowo
    print(label)
    label_text = subjects[label[0]]#mengakses array subjects untuk mendapat label text dari wajah yg sudah terprediksi
    print ("Subject pada gambar adalah\n"+label_text)
    draw_rectangle(img, rect)#drawing bound pada wajah yang terdeteksi
    draw_text(img, label_text, rect[0], rect[1]-5) #draw label text pada wajah yang terdeteksi
    return img, label_text

In [ ]:
#Proses Testing/Prediksi
print("Testing...")
#mengakses gambar yg akan diprediksi
test_img1 = cv2.imread("Data-Training/female/anpage/anpage.1.jpg") #gambar dari training data
test_img2 = cv2.imread("Data-Training/male/ajflem/ajflem1.jpg") #gambar dari training data
test_img3 = cv2.imread("test-data/prabowo_2.jpg") #gambar dari data test
test_img4 = cv2.imread("test-data/pakde_1.jpg") #gambar dari data test
#test_img5 = cv2.imread("data-test/CR7.jpg") #gambar dari data test yang tidak ada di training
#test_img6 = cv2.imread("data-test/Trump.jpg") #gambar dari data test yang tidak ada di training

print("Result:")
#memanggil fungsi testing
hasiltesting1, label_text1 = testing(test_img1)
#hasiltesting2, label_text2 = testing(test_img2)
hasiltesting3, label_text3 = testing(test_img3)
hasiltesting4, label_text4 = testing(test_img4)
#hasiltesting5, label_text5 = testing(test_img5)
#hasiltesting6, label_text6 = testing(test_img6)
#menampilkan hasil prediksi
cv2.imshow ((label_text1+" (dari training data)"), hasiltesting1)
#cv2.imshow ((label_text2+" (dari training data)"), hasiltesting2)
cv2.imshow ((label_text3+" (dari data test)"), hasiltesting3)
cv2.imshow ((label_text4+" (dari data test)"), hasiltesting4)
#cv2.imshow ((label_text5+" (wajah yang tidak ditraining)"), hasiltesting5)
#cv2.imshow ((label_text6+" (wajah yang tidak ditraining)"), hasiltesting6)

#mengakhiri program
cv2.waitKey(0)
cv2.destroyAllWindows()


Testing...
Result:
(1, 0.0)
Subject pada gambar adalah
Female
(2, 6006.810665676225)
Subject pada gambar adalah
Male
(2, 4477.687590210731)
Subject pada gambar adalah
Male
